In [1]:
pip install kafka-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 7.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
from kafka import KafkaProducer
from kafka.admin import KafkaAdminClient, NewTopic
from time import sleep
from json import dumps
import json

In [2]:
import yfinance as yf
import pandas as pd
from datetime import date,timedelta

In [3]:
admin_client = KafkaAdminClient(
    bootstrap_servers="<IP address>", 
    client_id='client_1'
)

In [21]:
topic = NewTopic(name="stock_market_analysis", num_partitions=1, replication_factor=1)
admin_client.create_topics(new_topics=[topic], validate_only=False)

CreateTopicsResponse_v3(throttle_time_ms=0, topic_errors=[(topic='stock_market_analysis', error_code=0, error_message=None)])

In [9]:
producer = KafkaProducer(
                bootstrap_servers=['<IP address>'],
                value_serializer=lambda x: 
                dumps(x).encode('utf-8')
)

In [5]:
# tickers
indices = ['NFLX','MSFT','AAPL','GOOGL','SNOW','TSLA','AMZN','META']

In [37]:
def get_daily_stock_data(indices):
    yesterday =  str(date.today() + timedelta(days = -1))
    today =  str(date.today())
    stock_data = pd.DataFrame()
    for index in indices:
        data = yf.download(tickers = index, start = yesterday, end = today)
        data = data.assign(stock_index = index)
        data = data.assign(date = data.index)
        stock_data = pd.concat([stock_data, data])
    return stock_data.to_json(orient = 'records'),stock_data

In [54]:
stock_data, st = get_daily_stock_data(indices)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [49]:
st.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'stock_index',
       'date'],
      dtype='object')

In [55]:
producer.send('stock_market_analysis', value = stock_data)

#  Load historical Data from Jan 2021 to Dec 2022

In [50]:
stock_data = pd.DataFrame()
for index in indices:
    data = yf.download(tickers = index, start = '2021-01-01' , end = '2022-12-30')
    data = data.assign(stock_index = index)
    data = data.assign(date = data.index)
    stock_data = pd.concat([stock_data, data])

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [51]:
stock_data

,Open,High,Low,Close,Adj Close,Volume,stock_index,date
Date,,,,,,,,
2021-01-04,539.000000,540.799988,515.090027,522.859985,522.859985,4444400,NFLX,2021-01-04
2021-01-05,521.549988,526.780029,515.890015,520.799988,520.799988,3133900,NFLX,2021-01-05
2021-01-06,511.970001,513.099976,499.500000,500.489990,500.489990,5346200,NFLX,2021-01-06
2021-01-07,508.279999,516.440002,506.420013,508.890015,508.890015,3686400,NFLX,2021-01-07
2021-01-08,511.309998,513.239990,504.510010,510.399994,510.399994,2973900,NFLX,2021-01-08
...,...,...,...,...,...,...,...,...
2022-12-22,117.199997,118.620003,114.379997,117.120003,117.120003,23618100,META,2022-12-22
2022-12-23,116.029999,118.180000,115.540001,118.040001,118.040001,17796600,META,2022-12-23
2022-12-27,117.930000,118.599998,116.050003,116.879997,116.879997,21392300,META,2022-12-27


In [52]:
producer.send('stock_market_analysis', value = stock_data.to_json(orient = 'records'))